# Description

This notebook calculates the probability that a second series has lower underlying discrete success rate than the first one. 
As input, we need two test runs results with discrete success test results 
`n_trials_1` `n_trials_2` `n_successes_1` `n_successes_2`. The calcation is based on a uniform prior and the assumption that the results of series1 and series2 are statistically independent and that the success rates within each series are constant.

Typically, this question arises when the percentage of successful trials in series2 was higher than in series1, i.e. when`n_successes_2/n_trials_2 > n_successes_1/n_trials_1`.
When faced with test results, the calculations tell you how often you should expect the test results in series2 to look better than in series1 despite the underlying success probability (which you would get for infinitely long series) being actually worse in series2. 

Note: all probabilities are given as numbers between 0 and 1, where 0 represents "never" and 1 represents 100% certainty.

# Parameters

In [1]:
n_successes_1 = 7 #number of successful trials in 1st series
n_trials_1 = 10 #number of trials conducted in 1st serie
n_successes_2 = 8 #number of successful trials in 2nd series
n_trials_2 = 10 #number of trials conducted in 2nd series

round_to_digits = 4

# Evaluation

In [2]:
from sympy import *
import numpy as np

def L(p, k, n): # non-normalized, integral is not 1
    #assert(n_successes <= n_trials and n_successes >= 0)
    return p**k * (1-p)**(n - k)

def P(p,k,n): #normalized, s.t. integral over p for given (k,n) is 1
    return simplify(L(p, k, n) / integrate(L(p, k, n), (p, 0, 1)))

#p = symbols('p', real=True, positive=True)
#n, k = symbols('n k', integer=True, positive=True)
#print('L(p, k, n) =', L(p, k, n))
#print('P(p, k, n) =', P(p, k, n))

p1, p2 = symbols('p1 p2', real=True, positive=True)
n1, k1, n2, k2 = symbols('n1 k1 n2 k2', integer=True, positive=True)

def P_product(p1,k1,n1, p2,k2,n2): # assuming statistically independent trial series here.
    return P(p1,k1,n1) * P(p2,k2,n2)

#print('P_product(p1,k1,n1, p2,k2,n2) =', P_product(p1,k1,n1, p2,k2,n2))

#print('norm L =', simplify(integrate(L(p, k, n), (p,0,1))))
#print('norm P =', simplify(integrate(P(p, k, n), (p,0,1))))
#print('norm P_product =', simplify(integrate(integrate(P_product(p1, k1, n1, p2, k2, n2), (p1, 0, 1)), (p2, 0, 1))))

In [ ]:
Pp = P_product(p1,k1,n1, p2,k2,n2)
Pprod_given = simplify(Pp.subs({k1:n_successes_1, n1:n_trials_1, k2:n_successes_2, n2:n_trials_2}))

int_0_p1 = simplify(integrate(Pprod_given,(p2,0,p1)))
#print(intp2ltp1)
P_p2ltp1 = integrate(int_0_p1, (p1,0,1))
#print(P_p2ltp1)
#print(P_p2ltp1.evalf())

print(f'You have conducted two test series. The first one had {n_successes_1} out of {n_trials_1} successful results, the seconds one had {n_successes_2} out of {n_trials_2} successful results.')
print(f'Using the assumptions, the probability is {round(P_p2ltp1.evalf(), round_to_digits)} to get a result where series2 has more successes than series1, even though the unknown underlying success rate within series2 is lower than that of series1. Thus, we are {1-round(P_p2ltp1.evalf(), round_to_digits)} confident that the single success rate p_success_2 is greater than p_success_1.')

In [ ]:
import matplotlib.pyplot as plt
from sympy.utilities.lambdify import lambdify

fxy = np.vectorize(lambdify([p1,p2], Pprod_given))

range_lo = 0.5 # adapt plot range here
range_hi = 1.0
xx, yy = np.meshgrid(np.arange(range_lo, range_hi, 1 / 100.), np.arange(range_lo, range_hi, 1 / 100.))
zz = fxy(xx,yy)

fig = plt.figure()

ax = fig.add_subplot(1,2,1)
ax.set_aspect('equal') # only available in 2D
ax.set_xlabel("p_success_1"); ax.set_ylabel("p_success_2"); 
ax.plot([range_lo,range_hi],[range_lo,range_hi], color='lightgreen',zorder=3)

c = ax.contourf(xx, yy, zz)
c = ax.contour(xx, yy, zz)
#ax.clabel(c, c.levels, inline=True, colors="c")

ax3 = fig.add_subplot(1,2,2, projection='3d')
ax3.set_xlabel("p_success_1"); ax3.set_ylabel("p_success_2"); 
ax3.set_title(f'probability density that a (p_success_1, p_success_2) tuple produces the given test results ({n_successes_1}/{n_trials_1} and {n_successes_2}/{n_trials_2})')
ax3.contourf(xx, yy, zz)
ax3.plot([range_lo,range_hi],[range_lo,range_hi], color='lightgreen',zorder=3)

print("Plotted in green is the diagonal.")
print("Below the diagonal is where p_success_2 is lower than p_success_1 and still the results look like series2 is more successful.")